In [7]:
import os
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

# Set up the Streamlit page configuration
st.set_page_config(page_title="NASDAQ Stock Dashboard", layout="wide")

# Caching metadata and data loading to speed up subsequent runs
@st.cache_data
def load_metadata():
    meta_path = os.path.join("../dataset", "symbols_valid_meta.csv")
    # Read the metadata CSV
    meta_df = pd.read_csv(meta_path)
    return meta_df

@st.cache_data
def load_stock_data(ticker, type_str):
    # Determine folder based on type: use "etfs" if type is ETF, otherwise "stocks"
    folder = "etfs" if type_str.lower() == "etf" else "stocks"
    file_path = os.path.join("../dataset", folder, f"{ticker}.csv")
    df = pd.read_csv(file_path)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df.sort_values("Date", inplace=True)
    return df

# Load metadata from dataset/symbols_valid_meta.csv
meta_df = load_metadata()

# Build dropdown options using the 'Symbol' and 'Security Name' columns.
# Also, derive the type: if 'ETF' is "Y" then type is ETF, otherwise Stock.
ticker_options = meta_df.apply(
    lambda row: f"{row['Symbol']} - {row['Security Name']}", axis=1
).tolist()

ticker_symbol_mapping = {
    f"{row['Symbol']} - {row['Security Name']}": (row['Symbol'], "ETF" if str(row['ETF']).strip().upper() == "Y" else "Stock")
    for _, row in meta_df.iterrows()
}

# Streamlit UI: Title and select box for ticker selection
st.title("NASDAQ Stock Dashboard")
selected_option = st.selectbox("Choose a stock/ETF:", ticker_options)

# Retrieve the ticker symbol and type from the selected option
ticker, type_str = ticker_symbol_mapping[selected_option]

# Display a subheader
st.subheader(f"{ticker} Price Data")

# Load the selected ticker's data
with st.spinner(f"Loading {ticker} data..."):
    df = load_stock_data(ticker, type_str)

# Create an interactive Plotly chart with a line for closing price and bars for volume
fig = go.Figure()

# Trace for closing price (line chart)
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Close"],
        mode="lines+markers",
        name="Close Price",
        line=dict(color="steelblue")
    )
)

# Trace for volume (bar chart on secondary y-axis)
fig.add_trace(
    go.Bar(
        x=df["Date"],
        y=df["Volume"],
        name="Volume",
        marker_color="lightgrey",
        opacity=0.5,
        yaxis="y2"
    )
)

# Update layout to include a secondary y-axis for volume
fig.update_layout(
    title=f"{ticker} Price and Volume",
    xaxis_title="Date",
    yaxis=dict(
        title="Close Price",
        side="left"
    ),
    yaxis2=dict(
        title="Volume",
        overlaying="y",
        side="right",
        showgrid=False
    ),
    hovermode="x unified",
    height=600,
    width=1000
)

# Display the Plotly chart in the Streamlit app
st.plotly_chart(fig, use_container_width=True)

2025-02-14 10:46:46.300 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 10:46:46.302 No runtime found, using MemoryCacheStorageManager
2025-02-14 10:46:46.303 No runtime found, using MemoryCacheStorageManager
2025-02-14 10:46:46.305 No runtime found, using MemoryCacheStorageManager
2025-02-14 10:46:46.306 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 10:46:46.306 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 10:46:46.307 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 10:46:46.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 10:46:46.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-14 10:46:46.46

DeltaGenerator()